In [50]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, TFBertForSequenceClassification
import pandas as pd

In [51]:
df = pd.read_excel("./DATA_FULL_TRAIN.xlsx")
df.head()

,Texto,Classe
0,SERVICO E LICENCAS DE SOFTWARE DE USO DO CALL CEN,MACHINERY
1,UPGRADE DO SISTEMA DE SEGURANCA DO SITE ALEXANDRE,MACHINERY
2,UPGRADE DO SISTEMA DE SEGURANCA DO SITE ALEXANDRE,MACHINERY
3,REFRIGERADOR 342 L FF CLEAN,MACHINERY
4,REFRIGERADOR 342 L FF CLEAN,MACHINERY


In [52]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

df["Classe"] = le.fit_transform(df["Classe"])

In [53]:
sentences = list(df["Texto"].values)
labels = list(df["Classe"].values)

In [55]:
model_name = "neuralmind/bert-base-portuguese-cased"
tokenizer = BertTokenizer.from_pretrained(model_name)

In [54]:
train_texts, val_texts, train_labels, val_labels = train_test_split(sentences, labels, test_size=0.2)

train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)

In [57]:
def encode_dataset(encodings, labels):
    return tf.data.Dataset.from_tensor_slices((
        dict(encodings),
        labels
    ))

train_dataset = encode_dataset(train_encodings, train_labels)
val_dataset = encode_dataset(val_encodings, val_labels)

In [ ]:
from transformers import TFBertForSequenceClassification

model = TFBertForSequenceClassification.from_pretrained(model_name, num_labels=6)

optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5, epsilon=1e-08)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

model.fit(train_dataset.shuffle(100).batch(16), epochs=8, batch_size=32)

In [59]:
loss, accuracy = model.evaluate(val_dataset.batch(64))
print(f"Loss: {loss}")
print(f"Accuracy: {accuracy}")

218/218 [==============================] - 352s 2s/step - loss: 0.1697 - accuracy: 0.9529
Loss: 0.16965046525001526
Accuracy: 0.9529149532318115
